## Youtube Data API

Add Youtube Functionality to your site

Search for content

Get the Youtube Channel ID: https://www.streamweasels.com/tools/youtube-channel-id-and-user-id-convertor/

## Features

- Control the video length

In [ ]:
!pip install --upgrade google-api-python-client
!pip install --upgrade google-auth-oauthlib google-auth-httplib2
!pip install schedule
!pip install ffmpeg-python
!pip install yt-dlp
!pip install unidecode

In [ ]:
import pandas as pd
import requests
import json

In [8]:
import os
import googleapiclient.discovery
from pprint import pprint 
import schedule
from datetime import datetime, timedelta
import time 
from Youtube import download_audio, download_video
from dotenv import load_dotenv

API_KEY = os.getenv("GOOGLE_API_KEY")
API_SERVICE_NAME = "youtube"
API_VERSION = "v3"

CHANNEL_ID = "UCabsTV34JwALXKGMqHpvUiA" # vtv24h
MAX_RESULT = 50

def get_channel_info():
    youtube = googleapiclient.discovery.build(API_SERVICE_NAME, API_VERSION, developerKey=API_KEY)
    request = youtube.channels().list(part="snippet", id=CHANNEL_ID)
    response = request.execute()

    pprint(response)

def get_channel_statistics():
    youtube = googleapiclient.discovery.build(API_SERVICE_NAME, API_VERSION, developerKey=API_KEY)
    request = youtube.channels().list(part="statistics", id=CHANNEL_ID)
    response = request.execute()

    pprint(response)

def create_video_url(video_id):
    return f"https://www.youtube.com/watch?v={video_id}"


def get_latest_video():
    print("Fetching latest video ")
    youtube = googleapiclient.discovery.build(API_SERVICE_NAME, API_VERSION, developerKey=API_KEY)

    yesterday = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d') + "T00:00:00Z"

    video_links = []

    try:
        # Fetch the latest video published after yesterday
        request = youtube.search().list(
            q="",
            channelId=CHANNEL_ID,
            maxResults=MAX_RESULT,
            order="date",
            publishedAfter=yesterday,
            type="video",
            part="snippet"
        )
        response = request.execute()
    
        print("Total number of results: ", response['pageInfo']['totalResults'])
    
        # Parse the items
        for item in response['items']:
            video_url = create_video_url(item['id']['videoId'])
            title = item['snippet']['title']
            desc = item['snippet']['description']
            publish_time = item['snippet']['publishedAt']
            print(video_url, title, desc, publish_time)
            video_links.append(video_url)
            
    except Exception as e:
        print(f"Error: {str(e)}")
    
    return response

    # print("Download videos")
    # download_video(video_links)

# Schedule the task to run every day at 7:00 AM 
# schedule.every().day.at("23:59").do(get_latest_video)
# schedule.every(60).seconds.do(get_latest_video)

# get_channel_statistics()
# response = get_latest_video()

# while True:
#     schedule.run_pending()
#     time.sleep(1)

In [9]:
response = get_latest_video()

Fetching latest video 
Total number of results:  37
https://www.youtube.com/watch?v=fmDoMNCZWRk Thời sự toàn cảnh tối 9/11: Vì sao phải tăng giá điện? | VTV24 Những nội dung sẽ có trong Thời sự toàn cảnh tối 9/11: 00:00 Tin chính 00:43 Làm khống hồ sơ nghiệm thu xe cơ giới 01:27 Bị ... 2023-11-09T14:00:11Z
https://www.youtube.com/watch?v=MT6czwLtz6A Giá bán lẻ điện bình quân tăng 4,5% | VTV24 Từ hôm nay, giá bán lẻ điện bình quân tăng 4,5%. Mức tăng giá này ảnh hưởng thế nào tới người thu nhập thấp, thu nhập trung ... 2023-11-09T12:02:49Z
https://www.youtube.com/watch?v=UB0Etn8h7Ek Nhiều cửa hàng bách hóa ở Thượng Hải giảm giá dịp lễ độc thân | VTV24 Ngày 11/11 là ngày độc thân ở Trung Quốc. Bốn số 1 được giải nghĩa giống như bốn cành cây trơ trụi, một cách diễn đạt dành ... 2023-11-09T10:00:21Z
https://www.youtube.com/watch?v=BlXKi8TokAU Câu chuyện quốc tế: Sống trong thành phố tương lai | VTV24 Từ việc tái sử dụng những công trình cũ, đến việc sử dụng những công nghệ giảm phát thải..

In [10]:
len(response['items'])

37

## Download Video/Audio

In [ ]:
from Youtube import download_audio, download_video
URLS = ["https://www.youtube.com/watch?v=mklY28SqTzc"]
download_audio(URLS)
download_video(URLS)

## Get Full Description of Video

In [55]:
import os
import googleapiclient.discovery
from pprint import pprint
import schedule
from datetime import datetime, timedelta
import time
from Youtube import download_audio, download_video
from dotenv import load_dotenv
from pprint import pprint
import re
from googleapiclient.errors import HttpError

API_KEY = os.getenv("GOOGLE_API_KEY")
API_SERVICE_NAME = "youtube"
API_VERSION = "v3"

CHANNEL_ID = "UCabsTV34JwALXKGMqHpvUiA"  # vtv24h
MAX_RESULT = 50


def get_channel_info():
    youtube = googleapiclient.discovery.build(
        API_SERVICE_NAME, API_VERSION, developerKey=API_KEY)
    request = youtube.channels().list(part="snippet", id=CHANNEL_ID)
    response = request.execute()

    pprint(response)


def get_channel_statistics():
    youtube = googleapiclient.discovery.build(
        API_SERVICE_NAME, API_VERSION, developerKey=API_KEY)
    request = youtube.channels().list(part="statistics", id=CHANNEL_ID)
    response = request.execute()

    pprint.print(response)


def create_video_url(video_id):
    return f"https://www.youtube.com/watch?v={video_id}"
def get_description_video(video_id):
    youtube = googleapiclient.discovery.build(
        API_SERVICE_NAME, API_VERSION, developerKey=API_KEY)
    desc = ""
    try:
        request = youtube.videos().list(
            id=video_id,
            part="snippet"
        )
        response = request.execute()
        for item in response['items']:
            desc = item['snippet']['description']
            print(response)

    except HttpError as error:
        print(f"Error: {error.content if error.content else str(error)}")
    return desc

def get_latest_video():
    print("Fetching latest video ")
    youtube = googleapiclient.discovery.build(
        API_SERVICE_NAME, API_VERSION, developerKey=API_KEY)

    yesterday = (datetime.now() - timedelta(days=1)
                 ).strftime('%Y-%m-%d') + "T00:00:00Z"

    video_links = []

    try:
        # Fetch the latest video published after yesterday
        request = youtube.search().list(
            q="",
            channelId=CHANNEL_ID,
            maxResults=MAX_RESULT,
            order="date",
            publishedAfter=yesterday,
            type="video",
            part="snippet"
        )
        response = request.execute()

        print("Total number of results: ",
              response['pageInfo']['totalResults'])

        # Parse the items
        for item in response['items']:
            video_url = create_video_url(item['id']['videoId'])
            title = item['snippet']['title']
            desc = item['snippet']['description']
            publish_time = item['snippet']['publishedAt']
            print(video_url, title, desc, publish_time)
            video_links.append(video_url)

    except Exception as e:
        print(f"Error: {str(e)}")

    return response

    # print("Download videos")
    # download_video(video_links)

# Schedule the task to run every day at 7:00 AM
# schedule.every().day.at("23:59").do(get_latest_video)
# schedule.every(60).seconds.do(get_latest_video)

# get_channel_statistics()
# response = get_latest_video()

# while True:
#     schedule.run_pending()
#     time.sleep(1)

In [56]:
desc = get_description_video("pS1YgHKZ4WI")

{'kind': 'youtube#videoListResponse', 'etag': 'COWvieOoe-tigtKvEUkCyAngL8Q', 'items': [{'kind': 'youtube#video', 'etag': '8Bs5Mb2d0q4jZxxlBns4fx1bXeg', 'id': 'pS1YgHKZ4WI', 'snippet': {'publishedAt': '2023-12-08T13:30:11Z', 'channelId': 'UCabsTV34JwALXKGMqHpvUiA', 'title': 'Thời sự toàn cảnh tối 8/12: Nhức nhối nạn phân biệt vùng miền trên mạng xã hội | VTV24', 'description': 'Thời sự toàn cảnh tối 8/12 sẽ có những nội dung sau:\n00:00 Tin chính\n00:52 Truy nã hai tội phạm bỏ trốn khỏi trại giam\n01:37 Bắt đối tượng lừa đầu tư bất động sản\n02:16 TP. Hồ Chí Minh: Miễn phí học phí cấp THCS\n02:53 Hà Nội sẽ có thêm thành phố thứ 3 ở phía Nam\n04:04 TP. Hồ Chí Minh: Lại xôn xao vụ nhân viên y tế bị hành hung\n06:05 Nhức nhối nạn phân biệt vùng miền trên mạng xã hội\n13:01 Bất an vì khỉ rừng tràn vào vườn nhà\n15:44 Lao đao vì vỡ hụi\n18:04 Người lao động mong chờ thưởng Tết Nguyên đán\n21:30 Can thiệp tim mạch cứu sống bệnh nhi sơ sinh\n21:52 Phẫu thuật tách rời cặp song sinh có chung lá 

In [57]:
{'kind': 'youtube#videoListResponse', 'etag': 'COWvieOoe-tigtKvEUkCyAngL8Q', 'items': [{'kind': 'youtube#video', 'etag': '8Bs5Mb2d0q4jZxxlBns4fx1bXeg', 'id': 'pS1YgHKZ4WI', 'snippet': {'publishedAt': '2023-12-08T13:30:11Z', 'channelId': 'UCabsTV34JwALXKGMqHpvUiA', 'title': 'Thời sự toàn cảnh tối 8/12: Nhức nhối nạn phân biệt vùng miền trên mạng xã hội | VTV24', 'description': 'Thời sự toàn cảnh tối 8/12 sẽ có những nội dung sau:\n00:00 Tin chính\n00:52 Truy nã hai tội phạm bỏ trốn khỏi trại giam\n01:37 Bắt đối tượng lừa đầu tư bất động sản\n02:16 TP. Hồ Chí Minh: Miễn phí học phí cấp THCS\n02:53 Hà Nội sẽ có thêm thành phố thứ 3 ở phía Nam\n04:04 TP. Hồ Chí Minh: Lại xôn xao vụ nhân viên y tế bị hành hung\n06:05 Nhức nhối nạn phân biệt vùng miền trên mạng xã hội\n13:01 Bất an vì khỉ rừng tràn vào vườn nhà\n15:44 Lao đao vì vỡ hụi\n18:04 Người lao động mong chờ thưởng Tết Nguyên đán\n21:30 Can thiệp tim mạch cứu sống bệnh nhi sơ sinh\n21:52 Phẫu thuật tách rời cặp song sinh có chung lá gan\n22:54 Phòng thí nghiệm khoa học ngầm sâu và lớn nhất thế giới\n23:39 CLB Santos lần đầu xuống hạng sau 111 năm\n25:03 Cụm tin quốc tế\n30:47 Chuyên gia tư vấn làm đẹp cao tuổi nhất thế giới\n----------\nĐồng hành cùng VTV Digital tại:\n\nỨng dụng VTVgo\nAndroid: https://bit.ly/305aQLs\niOS: https://apple.co/3g8yMTS\nhoặc xem trực tiếp trên https://vtvgo.vn/\n\nBáo điện tử: https://VTV.vn\n\nFanpage:\nTin tức, Xã hội: https://fb.com/tintucvtv24\nChuyên trang Tài Chính: https://fb.com/VTVMoney\nBáo điện tử VTV: https://fb.com/baodientuvtv\n\nYoutube: https://youtube.com/vtv24\nZalo: https://zalo.me/1571891271885013375\nInstagram: https://instagram.com/vtv24news/\nTiktok: tiktok.com/@vtv24news\nTwitter: https://twitter.com/vtvdigitalnews\n#VTVDigital #Tintuc', 'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/pS1YgHKZ4WI/default.jpg', 'width': 120, 'height': 90}, 'medium': {'url': 'https://i.ytimg.com/vi/pS1YgHKZ4WI/mqdefault.jpg', 'width': 320, 'height': 180}, 'high': {'url': 'https://i.ytimg.com/vi/pS1YgHKZ4WI/hqdefault.jpg', 'width': 480, 'height': 360}, 'standard': {'url': 'https://i.ytimg.com/vi/pS1YgHKZ4WI/sddefault.jpg', 'width': 640, 'height': 480}, 'maxres': {'url': 'https://i.ytimg.com/vi/pS1YgHKZ4WI/maxresdefault.jpg', 'width': 1280, 'height': 720}}, 'channelTitle': 'VTV24', 'tags': ['tin tức', 'vtv24', 'trung tâm tin tức vtv24', 'VTV Digital', 'tin tức vtv', 'thời sự vtv', 'thời sự vtv24', 'tin tức trong ngày', 'chuyển động 24h', 'tintuc', 'tin tuc', 'Thời sự toàn cảnh', 'tin tức tổng hợp', 'nhức nhối', 'nạn phân biệt vùng miền', 'phân biệt vùng miền trên mạng xã hội', 'mạng xã hội', 'phân biệt vùng miền', 'kỳ thi vùng miền', 'thời sự toàn cảnh vtv24', 'thời sự toàn cảnh vtv', 'parky', 'namkiki'], 'categoryId': '25', 'liveBroadcastContent': 'none', 'localized': {'title': 'Thời sự toàn cảnh tối 8/12: Nhức nhối nạn phân biệt vùng miền trên mạng xã hội | VTV24', 'description': 'Thời sự toàn cảnh tối 8/12 sẽ có những nội dung sau:\n00:00 Tin chính\n00:52 Truy nã hai tội phạm bỏ trốn khỏi trại giam\n01:37 Bắt đối tượng lừa đầu tư bất động sản\n02:16 TP. Hồ Chí Minh: Miễn phí học phí cấp THCS\n02:53 Hà Nội sẽ có thêm thành phố thứ 3 ở phía Nam\n04:04 TP. Hồ Chí Minh: Lại xôn xao vụ nhân viên y tế bị hành hung\n06:05 Nhức nhối nạn phân biệt vùng miền trên mạng xã hội\n13:01 Bất an vì khỉ rừng tràn vào vườn nhà\n15:44 Lao đao vì vỡ hụi\n18:04 Người lao động mong chờ thưởng Tết Nguyên đán\n21:30 Can thiệp tim mạch cứu sống bệnh nhi sơ sinh\n21:52 Phẫu thuật tách rời cặp song sinh có chung lá gan\n22:54 Phòng thí nghiệm khoa học ngầm sâu và lớn nhất thế giới\n23:39 CLB Santos lần đầu xuống hạng sau 111 năm\n25:03 Cụm tin quốc tế\n30:47 Chuyên gia tư vấn làm đẹp cao tuổi nhất thế giới\n----------\nĐồng hành cùng VTV Digital tại:\n\nỨng dụng VTVgo\nAndroid: https://bit.ly/305aQLs\niOS: https://apple.co/3g8yMTS\nhoặc xem trực tiếp trên https://vtvgo.vn/\n\nBáo điện tử: https://VTV.vn\n\nFanpage:\nTin tức, Xã hội: https://fb.com/tintucvtv24\nChuyên trang Tài Chính: https://fb.com/VTVMoney\nBáo điện tử VTV: https://fb.com/baodientuvtv\n\nYoutube: https://youtube.com/vtv24\nZalo: https://zalo.me/1571891271885013375\nInstagram: https://instagram.com/vtv24news/\nTiktok: tiktok.com/@vtv24news\nTwitter: https://twitter.com/vtvdigitalnews\n#VTVDigital #Tintuc'}, 'defaultAudioLanguage': 'vi'}}], 'pageInfo': {'totalResults': 1, 'resultsPerPage': 1}}


{'kind': 'youtube#videoListResponse',
 'etag': 'COWvieOoe-tigtKvEUkCyAngL8Q',
 'items': [{'kind': 'youtube#video',
   'etag': '8Bs5Mb2d0q4jZxxlBns4fx1bXeg',
   'id': 'pS1YgHKZ4WI',
   'snippet': {'publishedAt': '2023-12-08T13:30:11Z',
    'channelId': 'UCabsTV34JwALXKGMqHpvUiA',
    'title': 'Thời sự toàn cảnh tối 8/12: Nhức nhối nạn phân biệt vùng miền trên mạng xã hội | VTV24',
    'description': 'Thời sự toàn cảnh tối 8/12 sẽ có những nội dung sau:\n00:00 Tin chính\n00:52 Truy nã hai tội phạm bỏ trốn khỏi trại giam\n01:37 Bắt đối tượng lừa đầu tư bất động sản\n02:16 TP. Hồ Chí Minh: Miễn phí học phí cấp THCS\n02:53 Hà Nội sẽ có thêm thành phố thứ 3 ở phía Nam\n04:04 TP. Hồ Chí Minh: Lại xôn xao vụ nhân viên y tế bị hành hung\n06:05 Nhức nhối nạn phân biệt vùng miền trên mạng xã hội\n13:01 Bất an vì khỉ rừng tràn vào vườn nhà\n15:44 Lao đao vì vỡ hụi\n18:04 Người lao động mong chờ thưởng Tết Nguyên đán\n21:30 Can thiệp tim mạch cứu sống bệnh nhi sơ sinh\n21:52 Phẫu thuật tách rời cặp

In [ ]:
https://www.youtube.com/watch?v=pS1YgHKZ4WI&t=0s

https://www.youtube.com/watch?v=pS1YgHKZ4WI&t=52s

https://www.youtube.com/watch?v=pS1YgHKZ4WI&t=97s

In [54]:
response = get_latest_video()

Fetching latest video 
Total number of results:  26
https://www.youtube.com/watch?v=vtBZpZb2tHM Đồng Tháp: Hàng trăm người lao đao vì vỡ hụi hơn 30 tỷ đồng  | VTV24 Hàng chục tỷ đồng của những người tham gia hụi này có nguy cơ mất trắng khi chủ hụi tuyên bố vỡ hụi. ---------- Đồng hành cùng ... 2023-12-08T12:40:02Z
https://www.youtube.com/watch?v=vF9kR9jHpeE CLB của Pele, Neymar lần đầu xuống hạng sau 111 năm, CĐV tức giận gây bạo loạn | VTV24 Không thể chấp nhận thực tế rằng Santos phải xuống hạng lần đầu tiên sau 111 năm tồn tại, người hâm mộ đã gây ra cuộc bạo ... 2023-12-08T12:15:03Z
https://www.youtube.com/watch?v=KheJRqEbdW0 Phát lệnh truy nã 2 phạm nhân nguy hiểm bỏ trốn khỏi nơi giam giữ | VTV24 Sau hơn một ngày truy tìm song chưa có kết quả, Giám thị Trại giam Xuân Hà vừa ký quyết định truy nã 2 phạm nhân nguy hiểm. 2023-12-08T11:47:10Z
https://www.youtube.com/watch?v=nxH8BmZqFEw Bát nháo tình trạng học hộ, thi hộ ở giảng đường đại học | VTV24 Việc học hộ, thi hộ có thể khiến 

In [58]:
a = 'Thời sự toàn cảnh tối 8/12: Nhức nhối nạn phân biệt vùng miền trên mạng xã hội | VTV24'

'Thời sự toàn cảnh' in a

True